In [1]:
import requests

csv = requests.get('https://raw.githubusercontent.com/farooq-teqniqly/blog-databricks-for-noobs/master/flight_data.txt').text

In [2]:
print(csv)

In [3]:
dbutils.fs.mkdirs('/demo_data/')
dbutils.fs.put('/demo_data/flight_data.csv', csv)

Wrote 1200120 bytes.
Out[3]: True

In [4]:
dbutils.fs.rm('dbfs:/demo_data/flight_data.csv')

Out[2]: True

In [5]:
%fs
ls

path,name,size
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-results/,databricks-results/,0
dbfs:/demo_data/,demo_data/,0
dbfs:/farooq/,farooq/,0
dbfs:/tmp/,tmp/,0


In [6]:
flight_data_df = (spark.read
               .format('csv')
               .options(header='true', inferSchema='true', delimiter='|')
               .load('dbfs:/demo_data/flight_data.csv'))

In [7]:
flight_data_df = flight_data_df.drop('_c0', '_c17')

In [8]:
flight_data_df.show()

+----+-----+----+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
year|month| day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+----+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
2014| 12.0| 8.0| 658| -7| 935| -5| VX| N846VA|1780.0| SEA| LAX| 132| 954.0| 6| 58|
2014| 1.0|22.0| 1040| 5| 1505| 5| AS| N559AS| 851.0| SEA| HNL| 360| 2677.0| 10| 40|
2014| 3.0| 9.0| 1443| -2| 1652| 2| VX| N847VA| 755.0| SEA| SFO| 111| 679.0| 14| 43|
2014| 4.0| 9.0| 1705| 45| 1839| 34| WN| N360SW| 344.0| PDX| SJC| 83| 569.0| 17| 5|
2014| 3.0| 9.0| 754| -1| 1015| 1| AS| N612AS| 522.0| SEA| BUR| 127| 937.0| 7| 54|
2014| 1.0|15.0| 1037| 7| 1352| 2| WN| N646SW| 48.0| PDX| DEN| 121| 991.0| 10| 37|
2014| 7.0| 2.0| 847| 42| 1041| 51| WN| N422WN|1520.0| PDX| OAK| 90| 543.0| 8| 47|
2014| 5.0|12.0| 1655| -5| 1842| -18| VX| N361VA| 755.0| SEA| SFO| 98| 679.0| 16| 55|
2014| 4.0|19.0| 1236| -4| 1508| -7| AS| N309AS| 490.0| SEA| SAN| 135| 1050.0| 12| 36|
2014| 11.0|19.0| 1812| -3| 2352| -4| AS| N564AS| 26.0| SEA| ORD| 198| 1721.0| 18| 12|
2014| 11.0| 8.0| 1653| -2| 1924| -1| AS| N323AS| 448.0| SEA| LAX| 130| 954.0| 16| 53|
2014| 8.0| 3.0| 1120| 0| 1415| 2| AS| N305AS| 656.0| SEA| PHX| 154| 1107.0| 11| 20|
2014| 10.0|30.0| 811| 21| 1038| 29| AS| N433AS| 608.0| SEA| LAS| 127| 867.0| 8| 11|
2014| 11.0|12.0| 2346| -4| 217| -28| AS| N765AS| 121.0| SEA| ANC| 183| 1448.0| 23| 46|
2014| 10.0|31.0| 1314| 89| 1544| 111| AS| N713AS| 306.0| SEA| SFO| 129| 679.0| 13| 14|
2014| 1.0|29.0| 2009| 3| 2159| 9| UA| N27205|1458.0| PDX| SFO| 90| 550.0| 20| 9|
2014| 12.0|17.0| 2015| 50| 2150| 41| AS| N626AS| 368.0| SEA| SMF| 76| 605.0| 20| 15|
2014| 8.0|11.0| 1017| -3| 1613| -7| WN| N8634A| 827.0| SEA| MDW| 216| 1733.0| 10| 17|
2014| 1.0|13.0| 2156| -9| 607| -15| AS| N597AS| 24.0| SEA| BOS| 290| 2496.0| 21| 56|
2014| 6.0| 5.0| 1733| -12| 1945| -10| OO| N215AG|3488.0| PDX| BUR| 111| 817.0| 17| 33|
+----+-----+----+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
only showing top 20 rows

In [9]:
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [10]:
flight_data_df = flight_data_df.drop('_c0', 'c_17')
flight_data_df = flight_data_df.where(F.trim(F.col('dep_time')) != 'NA')
flight_data_df = flight_data_df.where(F.trim(F.col('dep_delay')) != 'NA')
flight_data_df = flight_data_df.where(F.trim(F.col('arr_time')) != 'NA')
flight_data_df = flight_data_df.where(F.trim(F.col('arr_delay')) != 'NA')
flight_data_df = flight_data_df.where(F.trim(F.col('air_time')) != 'NA')
flight_data_df = flight_data_df.where(F.trim(F.col('hour')) != 'NA')
flight_data_df = flight_data_df.where(F.trim(F.col('minute')) != 'NA')

In [11]:
flight_data_clean_df = (flight_data_df.withColumn("month", flight_data_df["month"].cast(T.IntegerType()))
  .withColumn("day", flight_data_df["day"].cast(T.IntegerType()))
  .withColumn("dep_time", F.trim(flight_data_df["dep_time"]).cast(T.IntegerType()))
  .withColumn("dep_delay", F.trim(flight_data_df["dep_delay"]).cast(T.IntegerType()))
  .withColumn("arr_time", F.trim(flight_data_df["arr_time"]).cast(T.IntegerType()))
  .withColumn("arr_delay", F.trim(flight_data_df["arr_delay"]).cast(T.IntegerType()))
  .withColumn("air_time", F.trim(flight_data_df["air_time"]).cast(T.IntegerType()))
  .withColumn("hour", F.trim(flight_data_df["hour"]).cast(T.IntegerType()))
  .withColumn("minute", F.trim(flight_data_df["minute"]).cast(T.IntegerType()))
  .withColumn("flight", flight_data_df["flight"].cast(T.IntegerType()))
  .withColumn("distance", F.trim(flight_data_df["distance"]).cast(T.IntegerType())))

In [12]:
flight_data_clean_df.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+----+
year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|_c17|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+----+
2014| 12| 8| 658| -7| 935| -5| VX| N846VA| 1780| SEA| LAX| 132| 954| 6| 58|null|
2014| 1| 22| 1040| 5| 1505| 5| AS| N559AS| 851| SEA| HNL| 360| 2677| 10| 40|null|
2014| 3| 9| 1443| -2| 1652| 2| VX| N847VA| 755| SEA| SFO| 111| 679| 14| 43|null|
2014| 4| 9| 1705| 45| 1839| 34| WN| N360SW| 344| PDX| SJC| 83| 569| 17| 5|null|
2014| 3| 9| 754| -1| 1015| 1| AS| N612AS| 522| SEA| BUR| 127| 937| 7| 54|null|
2014| 1| 15| 1037| 7| 1352| 2| WN| N646SW| 48| PDX| DEN| 121| 991| 10| 37|null|
2014| 7| 2| 847| 42| 1041| 51| WN| N422WN| 1520| PDX| OAK| 90| 543| 8| 47|null|
2014| 5| 12| 1655| -5| 1842| -18| VX| N361VA| 755| SEA| SFO| 98| 679| 16| 55|null|
2014| 4| 19| 1236| -4| 1508| -7| AS| N309AS| 490| SEA| SAN| 135| 1050| 12| 36|null|
2014| 11| 19| 1812| -3| 2352| -4| AS| N564AS| 26| SEA| ORD| 198| 1721| 18| 12|null|
2014| 11| 8| 1653| -2| 1924| -1| AS| N323AS| 448| SEA| LAX| 130| 954| 16| 53|null|
2014| 8| 3| 1120| 0| 1415| 2| AS| N305AS| 656| SEA| PHX| 154| 1107| 11| 20|null|
2014| 10| 30| 811| 21| 1038| 29| AS| N433AS| 608| SEA| LAS| 127| 867| 8| 11|null|
2014| 11| 12| 2346| -4| 217| -28| AS| N765AS| 121| SEA| ANC| 183| 1448| 23| 46|null|
2014| 10| 31| 1314| 89| 1544| 111| AS| N713AS| 306| SEA| SFO| 129| 679| 13| 14|null|
2014| 1| 29| 2009| 3| 2159| 9| UA| N27205| 1458| PDX| SFO| 90| 550| 20| 9|null|
2014| 12| 17| 2015| 50| 2150| 41| AS| N626AS| 368| SEA| SMF| 76| 605| 20| 15|null|
2014| 8| 11| 1017| -3| 1613| -7| WN| N8634A| 827| SEA| MDW| 216| 1733| 10| 17|null|
2014| 1| 13| 2156| -9| 607| -15| AS| N597AS| 24| SEA| BOS| 290| 2496| 21| 56|null|
2014| 6| 5| 1733| -12| 1945| -10| OO| N215AG| 3488| PDX| BUR| 111| 817| 17| 33|null|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+----+
only showing top 20 rows

In [13]:
flight_data_clean_df = flight_data_clean_df.drop('_c17')

In [14]:
flight_data_clean_df.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
2014| 12| 8| 658| -7| 935| -5| VX| N846VA| 1780| SEA| LAX| 132| 954| 6| 58|
2014| 1| 22| 1040| 5| 1505| 5| AS| N559AS| 851| SEA| HNL| 360| 2677| 10| 40|
2014| 3| 9| 1443| -2| 1652| 2| VX| N847VA| 755| SEA| SFO| 111| 679| 14| 43|
2014| 4| 9| 1705| 45| 1839| 34| WN| N360SW| 344| PDX| SJC| 83| 569| 17| 5|
2014| 3| 9| 754| -1| 1015| 1| AS| N612AS| 522| SEA| BUR| 127| 937| 7| 54|
2014| 1| 15| 1037| 7| 1352| 2| WN| N646SW| 48| PDX| DEN| 121| 991| 10| 37|
2014| 7| 2| 847| 42| 1041| 51| WN| N422WN| 1520| PDX| OAK| 90| 543| 8| 47|
2014| 5| 12| 1655| -5| 1842| -18| VX| N361VA| 755| SEA| SFO| 98| 679| 16| 55|
2014| 4| 19| 1236| -4| 1508| -7| AS| N309AS| 490| SEA| SAN| 135| 1050| 12| 36|
2014| 11| 19| 1812| -3| 2352| -4| AS| N564AS| 26| SEA| ORD| 198| 1721| 18| 12|
2014| 11| 8| 1653| -2| 1924| -1| AS| N323AS| 448| SEA| LAX| 130| 954| 16| 53|
2014| 8| 3| 1120| 0| 1415| 2| AS| N305AS| 656| SEA| PHX| 154| 1107| 11| 20|
2014| 10| 30| 811| 21| 1038| 29| AS| N433AS| 608| SEA| LAS| 127| 867| 8| 11|
2014| 11| 12| 2346| -4| 217| -28| AS| N765AS| 121| SEA| ANC| 183| 1448| 23| 46|
2014| 10| 31| 1314| 89| 1544| 111| AS| N713AS| 306| SEA| SFO| 129| 679| 13| 14|
2014| 1| 29| 2009| 3| 2159| 9| UA| N27205| 1458| PDX| SFO| 90| 550| 20| 9|
2014| 12| 17| 2015| 50| 2150| 41| AS| N626AS| 368| SEA| SMF| 76| 605| 20| 15|
2014| 8| 11| 1017| -3| 1613| -7| WN| N8634A| 827| SEA| MDW| 216| 1733| 10| 17|
2014| 1| 13| 2156| -9| 607| -15| AS| N597AS| 24| SEA| BOS| 290| 2496| 21| 56|
2014| 6| 5| 1733| -12| 1945| -10| OO| N215AG| 3488| PDX| BUR| 111| 817| 17| 33|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
only showing top 20 rows